# Load tests with K6


In [1]:
import os
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(override=True)

True

## 0. Lookup compatible configs


First, get IE compute instance options


In [2]:
from autobench.compute_manager import ComputeManager

In [3]:
compute_manager = ComputeManager()
compute_manager.options.head()

,vendor,vendor_status,region,region_label,region_status,id,accelerator,num_gpus,memory_in_gb,gpu_memory_in_gb,instance_type,instance_size,architecture,status,price_per_hour,num_cpus
0,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-t4-x1,gpu,1,15,16,nvidia-t4,x1,Nvidia T4,available,0.5,3
1,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-t4-x4,gpu,4,192,64,nvidia-t4,x4,Nvidia T4,available,3.0,46
2,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-a10g-x1,gpu,1,30,24,nvidia-a10g,x1,Nvidia A10G,available,1.0,6
3,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-a10g-x4,gpu,4,186,96,nvidia-a10g,x4,Nvidia A10G,available,5.0,46
4,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-a100-x1,gpu,1,145,80,nvidia-a100,x1,Nvidia A100,available,4.0,11


In [4]:
compute_manager.options.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   vendor            21 non-null     object 
 1   vendor_status     21 non-null     object 
 2   region            21 non-null     object 
 3   region_label      21 non-null     object 
 4   region_status     21 non-null     object 
 5   id                21 non-null     object 
 6   accelerator       21 non-null     object 
 7   num_gpus          21 non-null     int64  
 8   memory_in_gb      21 non-null     int64  
 9   gpu_memory_in_gb  21 non-null     int64  
 10  instance_type     21 non-null     object 
 11  instance_size     21 non-null     object 
 12  architecture      21 non-null     object 
 13  status            21 non-null     object 
 14  price_per_hour    21 non-null     float64
 15  num_cpus          21 non-null     int64  
dtypes: float64(1), int64(4), object(11)
memory usa

User specifies their desired inputs


In [6]:
VENDOR = "aws"
REGION = "us-east-1"
# REGION = "eu-west-1"
GPU_TYPES = ["nvidia-a10g", "nvidia-l4"]
GPU_TYPES = ["nvidia-a10g"]

In [7]:
possible_instances = compute_manager.get_instance_details(
    vendor=VENDOR, region=REGION, gpu_types=GPU_TYPES
)

In [8]:
len(possible_instances)

2

Then, check if model will work on any of the desired instances, and if so, get each TGI config


In [9]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

viable_instances = compute_manager.get_viable_instance_configs(
    model_id=model_id, instances=possible_instances
)

In [10]:
viable_instances

[{'tgi_config': TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=8192, max_input_length=6000, max_total_tokens=6144, num_shard=1, quantize=None, estimated_memory_in_gigabytes=23.01),
  'instance_config': ComputeInstanceConfig(id='aws-us-east-1-nvidia-a10g-x1', vendor='aws', vendor_status='available', region='us-east-1', region_label='N. Virginia', region_status='available', accelerator='gpu', num_gpus=1, memory_in_gb=30, gpu_memory_in_gb=24, instance_type='nvidia-a10g', instance_size='x1', architecture='Nvidia A10G', status='available', price_per_hour=1.0, num_cpus=6)},
 {'tgi_config': TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=32768, max_input_length=6000, max_total_tokens=6144, num_shard=4, quantize=None, estimated_memory_in_gigabytes=98.36),
  'instance_config': ComputeInstanceConfig(id='aws-us-east-1-nvidia-a10g-x4', vendor='aws', vendor_status='available', region='us-east-1', region_label='N. Virginia', regi

## 1. Deploy LLM with TGI on Inference Endpoints


In [42]:
from autobench.deployment import Deployment
from autobench.config import DeploymentConfig

In [44]:
viable_instance = viable_instances[0]
viable_instance

{'tgi_config': TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=8192, max_input_length=6000, max_total_tokens=6144, num_shard=1, quantize=None, estimated_memory_in_gigabytes=23.01),
 'instance_config': ComputeInstanceConfig(id='aws-us-east-1-nvidia-a10g-x1', vendor='aws', vendor_status='available', region='us-east-1', region_label='N. Virginia', region_status='available', accelerator='gpu', num_gpus=1, memory_in_gb=30, gpu_memory_in_gb=24, instance_type='nvidia-a10g', instance_size='x1', architecture='Nvidia A10G', status='available', price_per_hour=1.0, num_cpus=6)}

In [49]:
viable_instances[:1]

[{'tgi_config': TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=8192, max_input_length=6000, max_total_tokens=6144, num_shard=1, quantize=None, estimated_memory_in_gigabytes=23.01),
  'instance_config': ComputeInstanceConfig(id='aws-us-east-1-nvidia-a10g-x1', vendor='aws', vendor_status='available', region='us-east-1', region_label='N. Virginia', region_status='available', accelerator='gpu', num_gpus=1, memory_in_gb=30, gpu_memory_in_gb=24, instance_type='nvidia-a10g', instance_size='x1', architecture='Nvidia A10G', status='available', price_per_hour=1.0, num_cpus=6)}]

In [45]:
viable_instance["tgi_config"].env_vars

{'MAX_BATCH_PREFILL_TOKENS': '8192',
 'MAX_INPUT_LENGTH': '6000',
 'MAX_TOTAL_TOKENS': '6144',
 'NUM_SHARD': '1',
 'MODEL_ID': '/repository'}

In [46]:
deployment_config = DeploymentConfig(
    tgi_config=viable_instance["tgi_config"],
    instance_config=viable_instance["instance_config"],
)

In [47]:
deployment = Deployment(deployment_config)

# deployment = Deployment(
#     deployment_config, existing_endpoint_name="d30f3a82-95b9-44e8-b0ca-617d17cc"
# )

Creating inference endpoint...
Endpoint created successfully: https://p6t119srp8ybkzus.us-east-1.aws.endpoints.huggingface.cloud


In [15]:
deployment.deployment_id, deployment.endpoint.status, deployment.endpoint.url

('d30f3a82-95b9-44e8-b0ca-617d17cc',
 'running',
 'https://ehdndzfif3v55m3w.us-east-1.aws.endpoints.huggingface.cloud')

## 2. Prepare ShareGPT data for realisitic inference workload


In [16]:
from autobench.config import DataConfig
from autobench.data import BenchmarkDataset

data_config = DataConfig()
benchmark_dataset = BenchmarkDataset(data_config)
benchmark_dataset.build_data()

100%|█████████▉| 1999/2000 [00:00<00:00, 47239.06it/s]


In [17]:
data_config.file_path

'benchmark_data/data.json'

## 2. Run K6 Load Test


In [18]:
# If necessary, install the openai Python library by running
# pip install openai

import requests
from huggingface_hub import get_token


API_URL = f"{deployment.endpoint.url}" + "/v1/chat/completions"
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {get_token()}",
    "Content-Type": "application/json",
}


def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response


output = query(
    {
        "messages": [{"role": "user", "content": "How old is Statute of Liberty?"}],
        "max_tokens": 150,
        "model": "tgi",
    }
)

In [19]:
print(output.content)

b'{"object":"chat.completion","id":"","created":1725459405,"model":"/repository","system_fingerprint":"2.2.1-dev0-sha-6cb42f4","choices":[{"index":0,"message":{"role":"assistant","content":"The Statue of Liberty was dedicated on October 28, 1886. It was designed by French sculptor Fr\xc3\xa9d\xc3\xa9ric Auguste Bartholdi and was a gift from the people of France to the people of the United States.\\n\\nThe statue was built in France and then taken apart and shipped to New York Harbor, where it was reassembled on Bedloe\'s Island (now known as Liberty Island) in New York City. The statue was officially dedicated on October 28, 1886, in a ceremony presided over by President Grover Cleveland.\\n\\nSo, as of 2023, the Statue of Liberty is approximately 137 years old."},"logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":18,"completion_tokens":133,"total_tokens":151}}'


In [20]:
from autobench.runner import (
    K6ConstantArrivalRateExecutor,
    Scenario,
)

In [21]:
executor = K6ConstantArrivalRateExecutor(
    pre_allocated_vus=10, rate_per_second=1, duration="10s"
)

In [22]:
benchmark_dataset.file_path

'/Users/andrewreed/Documents/success_projects/auto-bench/benchmark_data/data.json'

In [23]:
scenario = Scenario(
    host=deployment.endpoint.url,
    executor=executor,
    data_file=benchmark_dataset.file_path,
    output_dir=os.path.abspath("../autobench/benchmark_results"),
)

In [24]:
scenario.run()

Preparing scenario 28883518-badd-4bf1-bcf3-e93317f9c0c6
Running scenario 28883518-badd-4bf1-bcf3-e93317f9c0c6

          /\      |‾‾| /‾‾/   /‾‾/   
     /\  /  \     |  |/  /   /  /    
    /  \/    \    |     (   /   ‾‾\  
   /          \   |  |\  \ |  (‾)  | 
  / __________ \  |__| \__\ \_____/ .io

     execution: local
        script: /var/folders/w0/6t9rxkj97rv47l9sc0q22yth0000gn/T/autobench_0qbcyi7u_k6_script.js
        output: json (/Users/andrewreed/Documents/success_projects/auto-bench/autobench/benchmark_results/scenario_28883518-badd-4bf1-bcf3-e93317f9c0c6/results.json)

     scenarios: (100.00%) 1 scenario, 10 max VUs, 40s max duration (incl. graceful stop):
              * load_test: 1.00 iterations/s for 10s (maxVUs: 10, gracefulStop: 30s)


running (01.0s), 01/10 VUs, 0 complete and 0 interrupted iterations
load_test   [  10% ] 01/10 VUs  01.0s/10s  1.00 iters/s

running (02.0s), 02/10 VUs, 0 complete and 0 interrupted iterations
load_test   [  20% ] 02/10 VUs  02.0s/10

In [26]:
scenario.scenario_id

'28883518-badd-4bf1-bcf3-e93317f9c0c6'

## Benchmark Runner

In [28]:
from autobench.runner import BenchmarkRunner

In [31]:
benchmark_runner = BenchmarkRunner(
    deployment=deployment,
    benchmark_dataset=benchmark_dataset,
    output_dir=os.path.abspath("../autobench/benchmark_results"),
)

In [32]:
deployment.endpoint.url

'https://ehdndzfif3v55m3w.us-east-1.aws.endpoints.huggingface.cloud'

In [43]:
deployment.endpoint.delete()

InferenceEndpoint(name='d30f3a82-95b9-44e8-b0ca-617d17cc', namespace='andrewrreed', repository='meta-llama/Meta-Llama-3-8B-Instruct', status='running', url='https://ehdndzfif3v55m3w.us-east-1.aws.endpoints.huggingface.cloud')

In [33]:
benchmark_runner.run_benchmark()

Running benchmark for deployment d30f3a82-95b9-44e8-b0ca-617d17cc
Running benchmark for arrival rate 1
Preparing scenario f2f19166-3e7d-4160-bcae-b05d55be8be1
Running scenario f2f19166-3e7d-4160-bcae-b05d55be8be1

          /\      |‾‾| /‾‾/   /‾‾/   
     /\  /  \     |  |/  /   /  /    
    /  \/    \    |     (   /   ‾‾\  
   /          \   |  |\  \ |  (‾)  | 
  / __________ \  |__| \__\ \_____/ .io

     execution: local
        script: /var/folders/w0/6t9rxkj97rv47l9sc0q22yth0000gn/T/autobench_au38ivak_k6_script.js
        output: json (/Users/andrewreed/Documents/success_projects/auto-bench/autobench/benchmark_results/deployment_d30f3a82-95b9-44e8-b0ca-617d17cc/scenario_f2f19166-3e7d-4160-bcae-b05d55be8be1/results.json)

     scenarios: (100.00%) 1 scenario, 50 max VUs, 35s max duration (incl. graceful stop):
              * load_test: 1.00 iterations/s for 5s (maxVUs: 50, gracefulStop: 30s)


running (00.9s), 01/50 VUs, 0 complete and 0 interrupted iterations
load_test   [  17% 

## Scheduler

In [34]:
vi = viable_instances[0]

In [38]:
vi["instance_config"].instance_type, vi["instance_config"].vendor, vi[
    "instance_config"
].num_gpus

('nvidia-a10g', 'aws', 1)

In [36]:
vi

{'tgi_config': TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=8192, max_input_length=6000, max_total_tokens=6144, num_shard=1, quantize=None, estimated_memory_in_gigabytes=23.01),
 'instance_config': ComputeInstanceConfig(id='aws-us-east-1-nvidia-a10g-x1', vendor='aws', vendor_status='available', region='us-east-1', region_label='N. Virginia', region_status='available', accelerator='gpu', num_gpus=1, memory_in_gb=30, gpu_memory_in_gb=24, instance_type='nvidia-a10g', instance_size='x1', architecture='Nvidia A10G', status='available', price_per_hour=1.0, num_cpus=6)}

In [39]:
viable_instances

[{'tgi_config': TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=8192, max_input_length=6000, max_total_tokens=6144, num_shard=1, quantize=None, estimated_memory_in_gigabytes=23.01),
  'instance_config': ComputeInstanceConfig(id='aws-us-east-1-nvidia-a10g-x1', vendor='aws', vendor_status='available', region='us-east-1', region_label='N. Virginia', region_status='available', accelerator='gpu', num_gpus=1, memory_in_gb=30, gpu_memory_in_gb=24, instance_type='nvidia-a10g', instance_size='x1', architecture='Nvidia A10G', status='available', price_per_hour=1.0, num_cpus=6)},
 {'tgi_config': TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=32768, max_input_length=6000, max_total_tokens=6144, num_shard=4, quantize=None, estimated_memory_in_gigabytes=98.36),
  'instance_config': ComputeInstanceConfig(id='aws-us-east-1-nvidia-a10g-x4', vendor='aws', vendor_status='available', region='us-east-1', region_label='N. Virginia', regi

In [27]:
from huggingface_hub.constants import INFERENCE_ENDPOINTS_ENDPOINT
from huggingface_hub.utils import get_session, hf_raise_for_status, build_hf_headers


def fetch_quotas(namespace: str):
    """
    Fetch quotas for a given namespace.

    Args:
        namespace (str): The namespace to fetch quotas for.

    Returns:
        dict: The quotas for the given namespace.
    """
    session = get_session()
    response = session.get(
        f"{INFERENCE_ENDPOINTS_ENDPOINT}/provider/quotas/{namespace}",
        headers=build_hf_headers(),
    )
    hf_raise_for_status(response)

    return response.json()

In [28]:
from huggingface_hub import whoami

In [29]:
whoami()

{'type': 'user',
 'id': '61d375fd733d3a83ecd1bba9',
 'name': 'andrewrreed',
 'fullname': 'Andrew Reed',
 'email': 'andrew.reed@huggingface.co',
 'emailVerified': True,
 'canPay': True,
 'periodEnd': 1727740799,
 'isPro': False,
 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/61d375fd733d3a83ecd1bba9/oIXwvvs1-HaCnJXMCZgkc.jpeg',
 'orgs': [{'type': 'org',
   'id': '5e67bd5b1009063689407478',
   'name': 'huggingface',
   'fullname': 'Hugging Face',
   'email': 'website@huggingface.co',
   'canPay': True,
   'periodEnd': 1727740799,
   'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/1583856921041-5dd96eb166059660ed1ee413.png',
   'roleInOrg': 'contributor',
   'isEnterprise': True},
  {'type': 'org',
   'id': '61f4051081c4d30f5814022f',
   'name': 'demo-org',
   'fullname': 'Demo Corp',
   'email': 'jeff@huggingface.co',
   'canPay': False,
   'periodEnd': 1727740799,
   'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/16

In [30]:
fetch_quotas("andrewrreed")

{'vendors': [{'name': 'aws',
   'quotas': [{'instanceType': 'intel-spr',
     'architecture': 'Intel Sapphire Rapids',
     'maxAccelerators': 40,
     'usedAccelerators': 0},
    {'instanceType': 'nvidia-a10g',
     'architecture': 'Nvidia A10G',
     'maxAccelerators': 8,
     'usedAccelerators': 3},
    {'instanceType': 'nvidia-t4',
     'architecture': 'Nvidia T4',
     'maxAccelerators': 15,
     'usedAccelerators': 0},
    {'instanceType': 'intel-icl',
     'architecture': 'Intel Ice Lake',
     'maxAccelerators': 40,
     'usedAccelerators': 0},
    {'instanceType': 'inf2',
     'architecture': 'AWS Inferentia 2',
     'maxAccelerators': 24,
     'usedAccelerators': 0},
    {'instanceType': 'nvidia-l4',
     'architecture': 'Nvidia L4',
     'maxAccelerators': 8,
     'usedAccelerators': 0},
    {'instanceType': 'nvidia-a100',
     'architecture': 'Nvidia A100',
     'maxAccelerators': 2,
     'usedAccelerators': 0}]},
  {'name': 'azure',
   'quotas': [{'instanceType': 'intel-xe

In [ ]:
{
    "vendors": [
        {
            "name": "aws",
            "quotas": [
                {
                    "instanceType": "nvidia-a10g",
                    "architecture": "Nvidia A10G",
                    "maxAccelerators": 8,
                    "usedAccelerators": 3,
                },
                {
                    "instanceType": "nvidia-l4",
                    "architecture": "Nvidia L4",
                    "maxAccelerators": 8,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "nvidia-a100",
                    "architecture": "Nvidia A100",
                    "maxAccelerators": 2,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "intel-icl",
                    "architecture": "Intel Ice Lake",
                    "maxAccelerators": 40,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "nvidia-t4",
                    "architecture": "Nvidia T4",
                    "maxAccelerators": 15,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "intel-spr",
                    "architecture": "Intel Sapphire Rapids",
                    "maxAccelerators": 40,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "inf2",
                    "architecture": "AWS Inferentia 2",
                    "maxAccelerators": 24,
                    "usedAccelerators": 0,
                },
            ],
        },
        {
            "name": "azure",
            "quotas": [
                {
                    "instanceType": "intel-xeon",
                    "architecture": "Intel Xeon",
                    "maxAccelerators": 40,
                    "usedAccelerators": 0,
                }
            ],
        },
        {
            "name": "gcp",
            "quotas": [
                {
                    "instanceType": "v5e",
                    "architecture": "Google Cloud TPU",
                    "maxAccelerators": 8,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "nvidia-l4",
                    "architecture": "Nvidia L4",
                    "maxAccelerators": 8,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "intel-spr",
                    "architecture": "Intel Sapphire Rapids",
                    "maxAccelerators": 40,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "nvidia-h100",
                    "architecture": "Nvidia H100",
                    "maxAccelerators": 2,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "nvidia-a100",
                    "architecture": "Nvidia A100",
                    "maxAccelerators": 2,
                    "usedAccelerators": 0,
                },
                {
                    "instanceType": "nvidia-t4",
                    "architecture": "Nvidia T4",
                    "maxAccelerators": 5,
                    "usedAccelerators": 0,
                },
            ],
        },
    ]
}

In [60]:
from autobench.scheduler import run_scheduler

In [61]:
viable_instances

[{'tgi_config': TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=8192, max_input_length=6000, max_total_tokens=6144, num_shard=1, quantize=None, estimated_memory_in_gigabytes=23.01),
  'instance_config': ComputeInstanceConfig(id='aws-us-east-1-nvidia-a10g-x1', vendor='aws', vendor_status='available', region='us-east-1', region_label='N. Virginia', region_status='available', accelerator='gpu', num_gpus=1, memory_in_gb=30, gpu_memory_in_gb=24, instance_type='nvidia-a10g', instance_size='x1', architecture='Nvidia A10G', status='available', price_per_hour=1.0, num_cpus=6)},
 {'tgi_config': TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=32768, max_input_length=6000, max_total_tokens=6144, num_shard=4, quantize=None, estimated_memory_in_gigabytes=98.36),
  'instance_config': ComputeInstanceConfig(id='aws-us-east-1-nvidia-a10g-x4', vendor='aws', vendor_status='available', region='us-east-1', region_label='N. Virginia', regi

In [62]:
namespace = "andrewrreed"
out = run_scheduler(viable_instances, namespace)

Running scheduler for 4 instances in namespace: andrewrreed


KeyboardInterrupt: 

Starting async scheduler with 4 instances for namespace: andrewrreed
Scheduler initialized with 4 viable instances for namespace: andrewrreed
Starting scheduler run
Updating quota information
Fetching quotas for namespace: andrewrreed
Quotas fetched successfully
Initializing tasks for 4 instances
Starting to process tasks
Checking if can deploy: aws nvidia-a10g (requires 1 GPUs)
Deployment possible: True
Checking if can deploy: aws nvidia-a10g (requires 4 GPUs)
Deployment possible: True
Checking if can deploy: aws nvidia-l4 (requires 4 GPUs)
Deployment possible: True
Checking if can deploy: aws nvidia-l4 (requires 1 GPUs)
Deployment possible: True
Current state: 0 pending tasks, 4 running tasks
Deploying and benchmarking instance: nvidia-a10g
Deploying and benchmarking instance: nvidia-a10g
Deploying and benchmarking instance: nvidia-l4
Deploying and benchmarking instance: nvidia-l4
Updating quota information
Fetching quotas for namespace: andrewrreed


Task exception was never retrieved
future: <Task finished name='Task-14' coro=<Scheduler.deploy_and_benchmark() done, defined at /Users/andrewreed/Documents/success_projects/auto-bench/autobench/scheduler.py:108> exception=AttributeError("'TGIConfig' object has no attribute 'tgi_config'")>
Traceback (most recent call last):
  File "/Users/andrewreed/Documents/success_projects/auto-bench/autobench/scheduler.py", line 112, in deploy_and_benchmark
    deployment = await asyncio.to_thread(
  File "/Users/andrewreed/.pyenv/versions/3.10.11/lib/python3.10/asyncio/threads.py", line 25, in to_thread
    return await loop.run_in_executor(None, func_call)
  File "/Users/andrewreed/.pyenv/versions/3.10.11/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/Users/andrewreed/Documents/success_projects/auto-bench/autobench/deployment.py", line 16, in __init__
    self.tgi_config = deployment_config.tgi_config
AttributeError: 'TGIConf

Quotas fetched successfully
Sleeping for 10 seconds before next check


In [47]:
import sys

In [48]:
sys.modules

{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module '_frozen_importlib' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 '_io': <module '_io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'posix': <module 'posix' (built-in)>,
 '_frozen_importlib_external': <module '_frozen_importlib_external' (frozen)>,
 'time': <module 'time' (built-in)>,
 'zipimport': <module 'zipimport' (frozen)>,
 '_codecs': <module '_codecs' (built-in)>,
 'codecs': <module 'codecs' from '/Users/andrewreed/.pyenv/versions/3.10.11/lib/python3.10/codecs.py'>,
 'encodings.aliases': <module 'encodings.aliases' from '/Users/andrewreed/.pyenv/versions/3.10.11/lib/python3.10/encodings/aliases.py'>,
 'encodings': <module 'encodings' from '/Users/andrewreed/.pyenv/versions/3.10.11/lib/python3.10/encodings/__init__.py'>,
 'e

aScheduler design:

Needs to:

1. Take in:
   - model_id
   - GPU types
   - Optionally, provider (AWS/GCP)
   - Optionally, specify (MAX_INPUT_TOKENS, MAX_TOTAL_TOKENS, MAX_BATCH_PREFILL_TOKENS, MAX_BATCH_TOTAL_TOKENS)
2. Determine which instances can fit the desired model
3. Iterate through instances to:
   a. deploy
   b. benchmark
   c. report results

Runner design:

1. Take in model_id, instance, TGI config
2. Deploy this
3. Shut down / pause instance
